In [5]:
# Import necessary libraries
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, degree

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]
print(f'Dataset: {dataset}:')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Number of features: {dataset.num_node_features}')
print(f'Number of classes: {dataset.num_classes}')

# Compute the normalized adjacency matrix for SGC
# Add self-loops to the adjacency matrix
edge_index, _ = add_self_loops(data.edge_index, num_nodes=data.num_nodes)
row, col = edge_index
# Compute degree and normalization factors
deg = degree(row, data.num_nodes, dtype=data.x.dtype)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
# Create sparse adjacency matrix
adj = torch.sparse_coo_tensor(edge_index, norm, torch.Size([data.num_nodes, data.num_nodes]))

# Precompute feature matrix for SGC (A^K * X)
K = 2  # Number of hops, matching two-layer GCN
x = data.x
for _ in range(K):
    x = torch.spmm(adj, x)  # Sparse matrix multiplication
feature_matrix = x

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Define the SGC model (linear version)
class SGC(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(SGC, self).__init__()
        self.linear = torch.nn.Linear(in_features, out_features)

    def forward(self, x):
        x = self.linear(x)
        return F.log_softmax(x, dim=1)

# Instantiate the models
gcn_model = GCN()
sgc_model = SGC(dataset.num_node_features, dataset.num_classes)

# Set up optimizers
optimizer_gcn = torch.optim.Adam(gcn_model.parameters(), lr=0.01, weight_decay=5e-4)
optimizer_sgc = torch.optim.Adam(sgc_model.parameters(), lr=0.01, weight_decay=5e-4)

# Training and evaluation functions for GCN
def train_gcn():
    gcn_model.train()
    optimizer_gcn.zero_grad()
    out = gcn_model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer_gcn.step()
    return loss.item()

def test_gcn():
    gcn_model.eval()
    out = gcn_model(data)
    pred = out.argmax(dim=1)
    acc = (pred[data.test_mask] == data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()
    return acc

# Training and evaluation functions for SGC
def train_sgc():
    sgc_model.train()
    optimizer_sgc.zero_grad()
    out = sgc_model(feature_matrix)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer_sgc.step()
    return loss.item()

def test_sgc():
    sgc_model.eval()
    out = sgc_model(feature_matrix)
    pred = out.argmax(dim=1)
    acc = (pred[data.test_mask] == data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()
    return acc

# Train GCN
print("\nTraining GCN...")
for epoch in range(200):
    loss = train_gcn()
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')
gcn_acc = test_gcn()
print(f'GCN Test Accuracy: {gcn_acc:.4f}')

# Train SGC
print("\nTraining SGC...")
for epoch in range(200):
    loss = train_sgc()
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')
sgc_acc = test_sgc()
print(f'SGC Test Accuracy: {sgc_acc:.4f}')

# Compare results
print(f"\nComparison:")
print(f"GCN Test Accuracy: {gcn_acc:.4f}")
print(f"SGC Test Accuracy: {sgc_acc:.4f}")

Dataset: Cora():
Number of nodes: 2708
Number of edges: 10556
Number of features: 1433
Number of classes: 7

Training GCN...
Epoch 0, Loss: 1.9464
Epoch 20, Loss: 0.2512
Epoch 40, Loss: 0.0693
Epoch 60, Loss: 0.0755
Epoch 80, Loss: 0.0426
Epoch 100, Loss: 0.0490
Epoch 120, Loss: 0.0478
Epoch 140, Loss: 0.0366
Epoch 160, Loss: 0.0287
Epoch 180, Loss: 0.0315
GCN Test Accuracy: 0.8160

Training SGC...
Epoch 0, Loss: 1.9506
Epoch 20, Loss: 0.4455
Epoch 40, Loss: 0.1728
Epoch 60, Loss: 0.1200
Epoch 80, Loss: 0.1057
Epoch 100, Loss: 0.0977
Epoch 120, Loss: 0.0912
Epoch 140, Loss: 0.0859
Epoch 160, Loss: 0.0817
Epoch 180, Loss: 0.0784
SGC Test Accuracy: 0.8000

Comparison:
GCN Test Accuracy: 0.8160
SGC Test Accuracy: 0.8000


In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import WikipediaNetwork
from torch_geometric.nn import GCNConv
from torch_geometric.utils import degree, to_undirected, add_self_loops
from pathlib import Path
import pickle

# -------------------------------
# Load Squirrel Dataset
# -------------------------------
root_dir = './data/Wikipedia'
dataset = WikipediaNetwork(root=root_dir, name='squirrel')
data = dataset[0]
print(f"Graph: {data}")

# -------------------------------
# Build Directed Edge List
# -------------------------------
raw_dir = Path(dataset.raw_dir)
graph_file = f'ind.{dataset.name.lower()}.graph'
graph_path = raw_dir / graph_file

if graph_path.exists():
    print(f"Loading graph from '{graph_file}'...")
    with open(graph_path, 'rb') as f:
        graph = pickle.load(f)
    directed_edges = [(src, nbr) for src, neighbors in graph.items() for nbr in neighbors]
    directed_edge_index = torch.tensor(directed_edges, dtype=torch.long).t().contiguous() if directed_edges else data.edge_index
    if not directed_edges:
        print("Warning: No edges found. Using processed edge_index as fallback.")
else:
    print(f"Graph file '{graph_file}' not found. Using data.edge_index.")
    directed_edge_index = data.edge_index

print(f"Directed edge_index shape: {directed_edge_index.shape}")

# -------------------------------
# Build Propagation Matrices
# -------------------------------
num_nodes = data.num_nodes

# **Directed Propagation Matrices** (for DirGCN and LinearDirGCN)
out_deg = degree(directed_edge_index[0], num_nodes, dtype=torch.float)
in_deg = degree(directed_edge_index[1], num_nodes, dtype=torch.float)
out_deg_inv_sqrt = out_deg.pow(-0.5)
in_deg_inv_sqrt = in_deg.pow(-0.5)
out_deg_inv_sqrt[out_deg_inv_sqrt == float('inf')] = 0
in_deg_inv_sqrt[in_deg_inv_sqrt == float('inf')] = 0
edge_weights = out_deg_inv_sqrt[directed_edge_index[0]] * in_deg_inv_sqrt[directed_edge_index[1]]
S_forward = torch.sparse_coo_tensor(directed_edge_index, edge_weights, (num_nodes, num_nodes)).coalesce()
S_backward = torch.sparse_coo_tensor(directed_edge_index[[1, 0]], edge_weights, (num_nodes, num_nodes)).coalesce()

# **Undirected Propagation Matrix** (for GCN and LinearGCN)
undirected_edge_index = to_undirected(directed_edge_index)
undirected_edge_index, _ = add_self_loops(undirected_edge_index, num_nodes=num_nodes)
deg = degree(undirected_edge_index[0], num_nodes, dtype=torch.float)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
edge_weights_undirected = deg_inv_sqrt[undirected_edge_index[0]] * deg_inv_sqrt[undirected_edge_index[1]]
S_undirected = torch.sparse_coo_tensor(undirected_edge_index, edge_weights_undirected, (num_nodes, num_nodes)).coalesce()

# -------------------------------
# Model Definitions
# -------------------------------

class DirGCNLayer(torch.nn.Module):
    """Nonlinear layer for directed GCN with separate weights for in/out edges."""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.W_forward = torch.nn.Linear(in_channels, out_channels, bias=False)
        self.W_backward = torch.nn.Linear(in_channels, out_channels, bias=False)

    def forward(self, x):
        x_forward = torch.spmm(S_forward, x)
        x_backward = torch.spmm(S_backward, x)
        return self.W_forward(x_forward) + self.W_backward(x_backward)

class DirGCN(torch.nn.Module):
    """Two-layer directed GCN."""
    def __init__(self):
        super().__init__()
        self.layer1 = DirGCNLayer(dataset.num_node_features, 16)
        self.layer2 = DirGCNLayer(16, dataset.num_classes)

    def forward(self, data):
        x = F.relu(self.layer1(data.x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.layer2(x)
        return F.log_softmax(x, dim=1)

class GCN(torch.nn.Module):
    """Two-layer GCN, agnostic to edge direction."""
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x = F.relu(self.conv1(data.x, data.edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, data.edge_index)
        return F.log_softmax(x, dim=1)

class LinearDirGCN(torch.nn.Module):
    """Linear version of DirGCN with separate weights for in/out edges."""
    def __init__(self, in_channels, out_channels, K=1):
        super().__init__()
        self.K = K
        self.lin_forward = torch.nn.Linear(in_channels, out_channels, bias=False)
        self.lin_backward = torch.nn.Linear(in_channels, out_channels, bias=False)

    def forward(self, data):
        x = data.x
        x_forward = x_backward = x
        for _ in range(self.K):
            x_forward = torch.spmm(S_forward, x_forward)
            x_backward = torch.spmm(S_backward, x_backward)
        out = self.lin_forward(x_forward) + self.lin_backward(x_backward)
        return F.log_softmax(out, dim=1)

class LinearGCN(torch.nn.Module):
    """Linear version of GCN, agnostic to edge direction."""
    def __init__(self, in_channels, out_channels, K=1):
        super().__init__()
        self.K = K
        self.linear = torch.nn.Linear(in_channels, out_channels)

    def forward(self, data):
        x = data.x
        for _ in range(self.K):
            x = torch.spmm(S_undirected, x)
        return F.log_softmax(self.linear(x), dim=1)

# -------------------------------
# Instantiate Models & Optimizers
# -------------------------------
models = {
    'DirGCN': DirGCN(),
    'GCN': GCN(),
    'LinearDirGCN': LinearDirGCN(dataset.num_node_features, dataset.num_classes),
    'LinearGCN': LinearGCN(dataset.num_node_features, dataset.num_classes)
}

optimizers = {name: torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4) 
              for name, model in models.items()}

# -------------------------------
# Training and Evaluation Functions
# -------------------------------
def train(model, optimizer, data):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask[:, 0]], data.y[data.train_mask[:, 0]])
    loss.backward()
    optimizer.step()
    return loss.item()

def test(model, data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    acc = (pred[data.test_mask[:, 0]] == data.y[data.test_mask[:, 0]]).sum().item() / data.test_mask[:, 0].sum().item()
    return acc

# -------------------------------
# Training Loops and Comparison
# -------------------------------
for name, model in models.items():
    print(f"\nTraining {name}...")
    optimizer = optimizers[name]
    for epoch in range(200):
        loss = train(model, optimizer, data)
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")
    acc = test(model, data)
    print(f"{name} Test Accuracy: {acc:.4f}")

Graph: Data(x=[5201, 2089], edge_index=[2, 217073], y=[5201], train_mask=[5201, 10], val_mask=[5201, 10], test_mask=[5201, 10])
Graph file 'ind.squirrel.graph' not found. Using data.edge_index.
Directed edge_index shape: torch.Size([2, 217073])

Training DirGCN...
Epoch 0, Loss: 1.6097
Epoch 20, Loss: 1.3986
Epoch 40, Loss: 1.2550
Epoch 60, Loss: 1.1661
Epoch 80, Loss: 1.1202
Epoch 100, Loss: 1.0719
Epoch 120, Loss: 1.0344
Epoch 140, Loss: 1.0119
Epoch 160, Loss: 0.9785
Epoch 180, Loss: 0.9896
DirGCN Test Accuracy: 0.4745

Training GCN...
Epoch 0, Loss: 1.6253
Epoch 20, Loss: 1.3987
Epoch 40, Loss: 1.2130
Epoch 60, Loss: 1.0959
Epoch 80, Loss: 1.0244
Epoch 100, Loss: 0.9603
Epoch 120, Loss: 0.9149
Epoch 140, Loss: 0.8841
Epoch 160, Loss: 0.8691
Epoch 180, Loss: 0.8431
GCN Test Accuracy: 0.2248

Training LinearDirGCN...
Epoch 0, Loss: 1.6094
Epoch 20, Loss: 1.2872
Epoch 40, Loss: 1.1680
Epoch 60, Loss: 1.1289
Epoch 80, Loss: 1.1136
Epoch 100, Loss: 1.1056
Epoch 120, Loss: 1.1011
Epoch 1

In [1]:
import numpy as np
from scipy.sparse import coo_matrix

def symmetrize_adj(mat):
    """
    Given a square COO sparse matrix 'mat', return a new matrix that is symmetric.
    For every edge (i, j) in 'mat', this adds an edge (j, i).
    """
    if mat.shape[0] != mat.shape[1]:
        raise ValueError("Input matrix must be square to symmetrize.")

    # Concatenate the original row and col with their counterparts swapped
    new_row = np.concatenate([mat.row, mat.col])
    new_col = np.concatenate([mat.col, mat.row])
    
    # If the original matrix has data, duplicate it. Otherwise, use 1's for edges.
    if mat.data is not None and len(mat.data) > 0:
        new_data = np.concatenate([mat.data, mat.data])
    else:
        new_data = np.ones(new_row.shape[0])
    
    # Create the symmetric COO matrix
    sym_mat = coo_matrix((new_data, (new_row, new_col)), shape=mat.shape)
    
    # Remove duplicate entries by summing them up (if any)
    sym_mat.sum_duplicates()
    return sym_mat

def main():
    # Create a directed adjacency matrix example:
    # Let's define a 4x4 matrix with edges:
    # 0 -> 1, 1 -> 2, 2 -> 3, 3 -> 0
    row = np.array([0, 1, 2, 3])
    col = np.array([1, 2, 3, 0])
    data = np.ones(len(row))
    adj = coo_matrix((data, (row, col)), shape=(4, 4))
    
    print("Original directed adjacency matrix (COO format):")
    print("Rows:", adj.row)
    print("Cols:", adj.col)
    print("Data:", adj.data)
    print("\nDense representation:")
    print(adj.toarray())

    # Apply symmetrization
    sym_adj = symmetrize_adj(adj)
    
    print("\nSymmetrized adjacency matrix (COO format):")
    print("Rows:", sym_adj.row)
    print("Cols:", sym_adj.col)
    print("Data:", sym_adj.data)
    print("\nDense representation:")
    print(sym_adj.toarray())

if __name__ == "__main__":
    main()


Original directed adjacency matrix (COO format):
Rows: [0 1 2 3]
Cols: [1 2 3 0]
Data: [1. 1. 1. 1.]

Dense representation:
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]]

Symmetrized adjacency matrix (COO format):
Rows: [0 0 1 1 2 2 3 3]
Cols: [1 3 0 2 1 3 0 2]
Data: [1. 1. 1. 1. 1. 1. 1. 1.]

Dense representation:
[[0. 1. 0. 1.]
 [1. 0. 1. 0.]
 [0. 1. 0. 1.]
 [1. 0. 1. 0.]]
